matplotlib practice

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import streamlit as st
import numpy as np
import pandas as pd
import openpyxl as op

In [2]:
mpl.__version__

'3.5.1'

In [3]:
# 세팅 불러오기
st.set_page_config(
    page_title="서울시 교통사고 현황",
    page_icon="Cars",
    #layout="wide",
)

In [4]:
st.markdown('<style>div.appview-container{background-color: #f7f6f4;}</style>',unsafe_allow_html=True)
st.markdown('<style>div[data-testid="stForm"]{background-color: #fcfbfb;}</style>',unsafe_allow_html=True)
st.markdown('<style>section[data-testid="stSidebar"]{background-color: #dfdedd;}</style>',unsafe_allow_html=True)

st.title("서울시 구별 교통사고 현황")

2023-01-15 16:43:28.511 
  command:

    streamlit run C:\Users\KRX\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [8]:
# ========== DATA & SETUP
# Load data
@st.cache
def load_data():
    df = pd.read_csv("https://raw.githubusercontent.com/ArubaKLM/Viz-Practice/main/road_accident_data.csv")
    return df

df = load_data()

InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `load_data()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function load_data at 0x00000278815FAEE0>
```

Please see the `hash_funcs` [documentation](https://docs.streamlit.io/library/advanced-features/caching#the-hash

In [9]:
# ========== SIDEBAR
# Customise map
st.sidebar.subheader("커스텀 디자인 만들기")

with st.form(key = 'Form1'):
    with st.sidebar:
        main_title = st.text_input("제목", "서울시 구별 교통사고 현황")
        title_fontsize = st.number_input("제목 크기", min_value=10, max_value=40, value=30)
        clr_title = st.color_picker('제목 색상', '#184e77')
        clr_background = st.color_picker('배경 색상', '#f7f6f4')
        clr_value = st.color_picker('수치 색상', '#1D4D79')
        clr_line = st.color_picker('선 색상', '#EC0808')
        clr_area = st.color_picker('영역 색상', '#F16322')
        alpha_area = st.number_input("투명도: 0(투명) ~ 1(불투명)",min_value=0.00, max_value=1.00, value=0.5)
        submitted = st.form_submit_button('적용하기')


In [ ]:
# =========== FILTER SELECTIONS

row_0, row_01 = st.columns([1,2])

with st.form(key='columns_in_form'):
    st.markdown('데이터 탐색하기')
    col1, col2 = st.columns(2)
    with col1:
        Harm = st.radio("사고 피해",('사고건수', '부상자수', '사망자수'))
    with col2:
        Category = st.radio("사고 유형", ('총계', '차-사람', '차-차', '차량단독'))
    submitted = st.form_submit_button('적용하기')
    # 기간 추가해서 조정할 수 있도록 해보자

footer = "원데이터: https://data.seoul.go.kr/dataList/322/S/2/datasetView.do \n 제작: 윤준식 | Lisa Hornung의 https://github.com/liloho/london-cycling-rates 프로젝트를 참고하여 제작함"
subtitle = "2007~2021년 서울시 자치구별" + Harm.lower() + Category.lower() # ex 차대차 사망건수

st.write("")


In [ ]:
# Dictionaries
display_dict = {"seoul":"서울시", "jong":"종로", "jungg":"중구", "yong":"용산", 
                "sungd":"성동", "gwang":"광진", "dongd":"동대문", "jungr":"중랑", 
                "sungb":"성북", "gangb":"강북", "dob":"도봉", "now":"노원", 
                "eun":"은평", "seod":"서대문", "ma":"마포", 
                "yang":"양천", "gangs":"강서", "gur":"구로", "gum":"금천", 
                "yeong":"영등포", "dongj":"동작", "gwan":"관악", 
                "seo":"서초", "gangn":"강남", "song":"송파", "gangd":"강동"}

In [ ]:
# =========== VISUALISATION

# Map view

## Plotting
#filter data set based on input

data = df[(df["district"].isin(display_dict.keys())) & (df["harm"]==Harm) & (df["category"]==Category)]
data["Display Name"] = data["district"].map(display_dict)


In [ ]:

# ========= Layout
# Initialise Figure and define layout
#8,7 | 10, 8.75
layout = [
        ["___","___","___","___","도봉","___","___","___"],
        ["___","___","은평","___","강북","노원","___","___"],
        ["___","___","서대문","종로","성북","동대문","중랑","___"],
        ["___","___","마포","중구","성동","광진","___","___"],
        ["___","___","___","용산","___","___","___","___"],
        ["강서","양천","영등포","___","서초","강남","송파","강동"],
        ["___","구로","금천","동작","___","___","___","___"],
        ["___","___","___","관악","___","___","___","___"],
        ]
fig,axs = plt.subplot_mosaic(layout, figsize=(12,9), empty_sentinel="___") 


In [ ]:

fig.set_facecolor(clr_background)
plt.subplots_adjust(wspace=0.1, hspace=0.1, left=0.05, right=0.95, bottom=0.05, top=0.9)



In [ ]:

#=========== Plotting
# 자치구 표시하기
y_values = ['2007', '2008', '2009','2010' ,'2011',
            '2012', '2013', '2014', '2015', '2016',
            '2017', '2018', '2019', '2020', '2021']        
x_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
height = max(data[y_values].max()) + 20

## Loop through all boroughs and map their values
for ax in axs:
    data_filtered = data[data["자치구"]==ax]
        
    #구의 이름 나타내기
    axs[ax].text(1.05, height-10, ax, fontsize=20, ha="left", va='top', color="#111111")
    
    #영역 나타내기
    axs[ax].fill_between(x_values, list(data_filtered[y_values].values[0]), zorder=1, color=clr_area, alpha=0.7,
                    linewidth=0)   
    axs[ax].plot(x_values, list(data_filtered[y_values].values[0]), zorder=2, 
             color=clr_line,linewidth=2.5)    
    
    #마지막 연도 점 나타내기
    axs[ax].scatter(max(x_values), list(data_filtered[y_values].values[0])[-1], zorder=3,color=clr_line)
    
    #최신 연도의 수치 나타내기
    axs[ax].text(max(x_values)-0.05, height-7, '{:<10d}'.format(list(data_filtered[y_values].values[0])[-1]), 
             ha="right", fontsize=20, fontweight='bold', va='top', color=clr_value)
    
    #배경 색 설정
    axs[ax].set_xlim(xmin=0.8, xmax=15.2)
    axs[ax].set_ylim(ymax=height, ymin=0) 
    axs[ax].set_facecolor(clr_background)    
    axs[ax].axis("off")

#============= 범례           
#add legend
lgd = fig.add_axes([0.8, 0.05, 0.1, 0.1]) #axes to hold legend
lgd.text(1.05,height-3,data["자치구"][0], fontsize=8, ha="left", va='top', color="#111111")
lgd.fill_between(x_values,list(data.loc[0][y_values].values), zorder=1,color="#999999", alpha=0.7,linewidth=0)    
lgd.plot(x_values,list(data.loc[0][y_values].values), zorder=2,color="#333333",linewidth=1.5)    
lgd.scatter(max(x_values),list(data.loc[0][y_values].values)[-1], zorder=3,color="#333333")
lgd.text(max(x_values)-0.05, height-2, '{:,.0f}%'.format(list(data.loc[0][y_values].values)[-1]), 
        ha="right", fontsize=15, fontweight='bold', va='top', color="#333333")
lgd.set_xlim(xmin=0.8, xmax=6.2)
lgd.set_ylim(ymax=height, ymin=0) 
lgd.set_facecolor("#E4E4E4")
for pos in ["top", "bottom", "right", "left"]:
    lgd.spines[pos].set_visible(False)
lgd.set_xticks([1,6], ["2007", "2021"],fontsize = 10)
lgd.set_yticks([])
lgd.annotate('2021년\n수치', xy=(max(x_values)+0.3, height-10), xycoords='data', xytext=(10, 0), textcoords='offset points', 
                   fontsize=11, ha='left', va='center', annotation_clip=False,
                    arrowprops=dict(arrowstyle="->",facecolor='black'))
lgd.annotate('자치구', xy=(min(x_values)+0.2, height-1), xycoords='data', xytext=(0, 16), textcoords='offset points', 
                   fontsize=11, ha='center', va='center', annotation_clip=False,
                    arrowprops=dict(arrowstyle="->",facecolor='black'))



#=============== Text           
## Add titles and footer
y_pos = 1.05
x_pos = 0.05

fig.text(x_pos, y_pos, main_title, fontsize=title_fontsize, ha='left',va="top",
             fontweight="bold",  color=clr_title)
fig.text(x_pos, y_pos-(title_fontsize*0.2*0.01), subtitle, fontsize=15, ha='left',va="top",
             fontweight="normal",   color="#111111")
fig.text(x_pos, -0.05, footer, fontsize=11, ha='left',va="center",
             fontweight="normal",  linespacing=1.5, color="#111111")


#============ 서울시 전체
#inner = df[(df["Area name"]=="Inner London") & (df["Frequency"]==frequency) & (df["Purpose"]==purpose)]["2021"].iloc[0]
#outer = df[(df["Area name"]=="Outer London") & (df["Frequency"]==frequency) & (df["Purpose"]==purpose)]["2021"].iloc[0]
#강동
#강북
#서북
#도심
서울시 = df[(df["자치구"]=="서울특별시") & (df["사고유형"]==Category) & (df["사고유형"]==Harm)]["2021"].iloc[0]

fig.text(x_pos, y_pos-0.14,  "2021년 전체: " + ''.format(서울시), fontsize=15, ha='left',va="top",
         fontweight="bold",color=clr_value)
fig.text(x_pos, y_pos-0.18, "Inner: " + '{:,.0f}%'.format(inner) + "  |  Outer: " + '{:,.0f}%'.format(outer) , 
         fontsize=11, ha='left',va="top", fontweight="regular",color="#111111")

st.pyplot(fig)


## ======= Download
st.write("")
st.write("")

# download data
csv = data[['ONS Code', 'Area name','2016', '2017', '2018','2019', '2020', '2021']].to_csv(index=False)
#목록 열거

st.download_button(
    label="CSV로 데이터 내려받기",
    data=csv,
    file_name='seoul_car_accident_%s_%s.csv' % (Harm.lower(), Category.lower()),
    mime='text/csv',
)

#download image
plt.savefig("seoul_car_accident.png",bbox_inches="tight", pad_inches=0.2)
with open("london_cycling_rates.png", "rb") as file:
    btn = st.download_button(
            label="이미지 저장하기",
            data=file,
            file_name="london_cycling_rates.png",
            mime="image/png"
          )

st.write("")
st.subheader("About")
st.markdown("Data source: [Active Lives Survey 2021](https://www.gov.uk/government/statistics/walking-and-cycling-statistics-england-2021)")
st.markdown("App created by Lisa Hornung. Visit my [website](https://inside-numbers.com/) or follow me on [Github](https://github.com/Lisa-Ho), [Mastodon](https://fosstodon.org/@LisaHornung), [Twitter](https://twitter.com/LisaHornung_).")

In [ ]:
plt.style.use("seaborn-notebook")

In [ ]:
fig = plt.figure()
ax = plt.axes()

In [ ]:
fig = plt.figure()
plt.plot(0, 0.2, 0.4, 0.6, 0.8, 1) * 5

In [ ]:
x= np.arange(0,10,0.01)
fit = plt.figure()
plt.plot(x, np.sin(x))
plt.plot(x, np.cos(x))
plt.plot(np.random.randn(50).cumsum())

In [ ]:
plt.plot(np.random.randn(50).cumsum(), linestyle='-')
plt.plot(np.random.randn(50).cumsum(), linestyle='dashed')
plt.plot(np.random.randn(50).cumsum(), linestyle='dashdot')
plt.plot(np.random.randn(50).cumsum(), linestyle=':')

플롯 축

In [ ]:
plt.plot(np.random.randn(50).cumsum(), linestyle='dotted')
plt.xlim(-1,50)
plt.ylim(-1,50)
plt.axis('tight')

In [ ]:
plt.plot(np.random.randn(50), label='A')
plt.plot(np.random.randn(50), label='B')
plt.axis('equal')
plt.xlabel("x")
plt.ylabel("randon.randm")

In [ ]:
font1 = { 'family': 'DejaVu Sans', 'size' : '24', 'color' : 'darkblue'}
font2 = { 'family': 'Liberation Mono', 'size' : '24', 'weight': 'bold', 'color' : 'black'}
font3 = { 'family': 'STIXGeneral', 'size': 16, 'weight' : 'light'}

plt.plot([1,2,4], [1,2,6])
plt.title('title', fontdict=font1)
plt.xlabel('x', fontdict=font2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.random.randn(10), '-r', label='A')
ax.plot(np.random.randn(10), '--b', label='B')
ax.legend(loc= 'upper center', frameon = False, ncol=2 )

In [ ]:
ax.legend(loc= 'upper center', fancybox= True, framealpha= .3, ncol=2, )

In [ ]:
x = np.linspace(0, 10, 1000)
y = np.cos(x[:, np.newaxis] * np.arange(0, 2, 0.2))
lines = plt.plot(x, y)
plt.legend(lines[:3], ['c1', 'c2', 'c3'])


In [ ]:
set([f.name for f in mpl.font_manager.fontManager.ttflist])

In [ ]:
plt.plot(x , y[:,0], label = 'c1')
plt.plot(x , y[:,1], label = 'c2')
plt.plot(x , y[:,2], label = 'c3')
plt.plot(x , y[:,3:], label = 'c4')
plt.legend(framealpha=1, frameon= True)

In [ ]:
x= np.linspace(0,20,100)
I = np.cos(x) - np.cos(x[:, np.newaxis])

plt.imshow(I, cmap='RdBu')
plt.colorbar();
plt.clim(-1,1);

In [ ]:
plt.imshow(I, cmap=plt.cm.get_cmap('Blues', 5))
plt.colorbar();
plt.clim(-1,1);

In [ ]:
plt.axes()
plt.axes([0.5,0.5,0.1,0.2])

In [ ]:
for i in range (1,10):
    plt.subplot(3,3,i)
    plt.text(0.5,0.5, str((3,3,i)), na='center')

In [ ]:
fig, ax = plt.subplots(3,3, sharex='col', sharey= 'row')

In [ ]:
grid = plt.GridSpec(2,3, wspace=0.4, hspace=0.4)

plt.subplot(grid[0,0])
plt.subplot(grid[0,1:])
plt.subplot(grid[1,:2])
plt.subplot(grid[1,2]);

In [ ]:
plt.figure(figsize=(5,6))

x=range(1,21);
columns = [np.random.randn(20) * i for i in range(1, 7)]

i = 0
for c in columns:
    i +=1
    
    plt.subplot(3,2,i)
    plt.plot(x, c, marker='o', linewidth=1, label=c)
    plt.xlim(-1,21)
    plt.ylim(c.min()-1, c.max()+1)